# Exercise 18

## Analyze class homeworks


In [1]:
import pandas as pd
import numpy as np


import pandas as pd
import numpy as np
import scipy as sp
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
# from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer

%matplotlib inline
import matplotlib.pyplot as plt

data = pd.read_excel('https://raw.githubusercontent.com/albahnsen/PracticalMachineLearningClass/master/datasets/E18.xlsx')
data.head()




,Sexo,T1,T2,T3,T4,T5,T6
0,H,Un científico de datos y un ingeniero de datos...,NaN,Los árboles de decisión es uno de los algoritm...,NaN,"En el documento ""Do We Need Hundreds of Classi...",Boosting builds models from individual so call...
1,H,Aunque los dos perfiles cuentan con habilidade...,Aunque el conceso no es total en el tema de ma...,NaN,En Machine Learing el propósito de los métodos...,El paper publicado en 2014 hace una evaluación...,Gradient Boosting ClassifierGradient Boosting ...
2,H,Python vs Stata. Analizar información es un pr...,En el marco del Machine Learning encontramos c...,Árbol de clasificación Como su nombre lo indic...,Este es un mecanismo que utiliza varios algori...,El paper analiza 179 clasificadores de 17 dife...,Gradient Boosting Classifier busca aprender de...
3,M,Los algoritmos de machine learning buscan pred...,Los algoritmos de machine learning buscan pred...,Los arboles de decisión pertenecen a los algor...,Consideremos el siguiente ejemplo: Una persona...,¿Se necesitan cientos de modelos para resolver...,NaN
4,H,Python vs R para análisis de datos. A pesar de...,Tipos de aprendizaje en Machine Learning. El M...,Tipos de árboles de decisión y aplicaciones. U...,La combinación de modelos o ensemble es un mod...,En el articulo se analizan cerca de 180 clasif...,XGBoosting vs GBoosting. A pesar de que el alg...


# Exercise 18.1

Analyze the writing patterns of each student

In [2]:
data.isna().sum()

Sexo     0
T1       1
T2      12
T3      12
T4       6
T5       7
T6      13
dtype: int64

In [0]:
data2=data.fillna(' ', inplace=False)

In [4]:
data2.isna().sum()

Sexo    0
T1      0
T2      0
T3      0
T4      0
T5      0
T6      0
dtype: int64

In [0]:
data2["Total"] = data2["T1"]+data2["T2"]+data2["T3"]+data2["T4"]+data2["T5"]+data2["T6"]

In [0]:
X = data2.Total

In [0]:
vect = CountVectorizer()
X_dtm = vect.fit_transform(X)

In [0]:
temp=X_dtm.todense()

In [9]:
df=pd.DataFrame.from_dict(vect.vocabulary_, orient='index',columns=['Number'])#['sap']
df.sort_values(by='Number', ascending=False)

,Number
útiles,5891
útil,5890
únicos,5889
único,5888
únicamente,5887
única,5886
últimos,5885
último,5884
últimas,5883
última,5882


# Exercise 18.2

Evaluate the similarities of the homeworks of the students

tip: https://github.com/orsinium/textdistance

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer().fit_transform(data2.Total)
tfidf

<42x5892 sparse matrix of type '<class 'numpy.float64'>'
	with 23210 stored elements in Compressed Sparse Row format>

In [38]:
from sklearn.metrics.pairwise import linear_kernel

cosine_similarities = linear_kernel(tfidf[0:1], tfidf).flatten()
cosine_similarities

array([1.        , 0.7080319 , 0.68806194, 0.68315145, 0.68733982,
       0.67198529, 0.43845454, 0.72022712, 0.69667547, 0.7163757 ,
       0.67971295, 0.78581136, 0.70872005, 0.68319567, 0.5935448 ,
       0.45033204, 0.68008225, 0.7323431 , 0.70962727, 0.67783596,
       0.68909146, 0.72949524, 0.65084057, 0.70823146, 0.65092852,
       0.68325451, 0.7235075 , 0.65436555, 0.64971361, 0.7129934 ,
       0.68927874, 0.67285875, 0.69888887, 0.73383415, 0.72820417,
       0.70363278, 0.69170822, 0.72935159, 0.24073437, 0.69601001,
       0.71585408, 0.71456039])

In [39]:

related_docs_indices = cosine_similarities.argsort()[:-5:-1]
related_docs_indices

array([ 0, 11, 33, 17])

# Exercise 18.3

Create a classifier to predict the sex of each student



In [23]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(X_dtm, data2['Sexo'], test_size=0.33, random_state=42)

clf = RandomForestClassifier(n_jobs=-1,n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [0]:
y_pred = clf.predict(X_test)

In [26]:
from sklearn import metrics
print(metrics.accuracy_score(y_test,y_pred))

0.9285714285714286


Son pocas muestras para que el modelo aprenda de las pocas mujeres que hay en los datos